In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from glob import glob
import multiprocessing
import pandas as pd
from tqdm import tqdm
import time
from copy import deepcopy
from multiprocessing import Pool
from hydra import initialize, compose
import pickle

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='../../config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.lens import Lens
from mejiro.plots import diagnostic_plot, plot
from mejiro.utils import util
from mejiro.scripts import generate
from mejiro.helpers import pyhalo

In [3]:
def add_subhalos(lens):
    from mejiro.helpers import pyhalo
    # add CDM subhalos
    try:
        lens.add_subhalos(*pyhalo.unpickle_subhalos('/data/bwedig/roman-pandeia/output/pickles/cdm_subhalos_tuple'))  # TODO hard-code path for now
        return lens
    except:
        return None

In [4]:
with initialize(version_base=None, config_path='../../config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, repo_dir, pickle_dir = config.machine.array_dir, config.machine.data_dir, config.machine.repo_dir, config.machine.pickle_dir
util.create_directory_if_not_exists(pickle_dir)

In [5]:
# open pickled lens list
with open(os.path.join(pickle_dir, '01_skypy_output_lens_list'), 'rb') as results_file:
    lens_list = pickle.load(results_file)

In [6]:
# split up the lenses into batches based on core count
cpu_count = multiprocessing.cpu_count()
process_count = int(cpu_count / 2)  # TODO consider making this larger, but using all CPUs has crashed
generator = util.batch_list(lens_list, process_count)
batches = list(generator)

In [7]:
# process the batches
updated_lenses = []
for batch in tqdm(batches):
    pool = Pool(processes=process_count)
    for updated_lens in pool.map(add_subhalos, batch):
        if updated_lens is not None:
            updated_lenses.append(updated_lens)

100%|██████████| 5/5 [00:01<00:00,  3.91it/s]


In [8]:
# pickle lens list
pickle_target = os.path.join(pickle_dir, '02_skypy_output_lens_list_with_subhalos')
util.delete_if_exists(pickle_target)
with open(pickle_target, 'ab') as results_file:
    pickle.dump(updated_lenses, results_file)